In [1]:
import sys

sys.path.append('/home/michieletto/hmp_utils')
if '../' not in sys.path:
    sys.path.append('../')

In [2]:
import json
import glob
import os
import random

from hmp_utils.motion.body_models_constants import SmplConstants
from hmp_utils.motion.mocap_processor import MocapLoader
from hmp_utils.motion.kinematics import ForwardKinematics
from hmp_utils.visualize.stickman_animation import pose_animation, CameraOrientation, AnimationMode

import torch
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import tqdm
import numpy as np
import pandas as pd
import wandb
import itertools
import time

In [3]:
dataset = []

# Definisci il percorso completo del file JSON
percorso_file_train = '/home/michieletto/datasets/BABEL_DATASET/babel_v1.0_release/train.json'
percorso_file_val   = '/home/michieletto/datasets/BABEL_DATASET/babel_v1.0_release/val.json'
percorso_file_test  = '/home/michieletto/datasets/BABEL_DATASET/babel_v1.0_release/test.json'

dataset.append(percorso_file_train)
dataset.append(percorso_file_val)
dataset.append(percorso_file_test)

data = {}           # Dizionario contenente l'intero dataset  

# Apre il file JSON in modalità lettura
for i,percorso_file in enumerate(dataset):
    with open(percorso_file, 'r') as f:
        # Carica i dati JSON
        data.update(json.load(f))


In [4]:

occur_lab = {
    'label': [],              
    'feat_p': []
    }

no_lab = 0

folders=[]
# Creazione del dataset contenente tutte le raw_label dei frame e passaggi cartelle
for i, k in enumerate(data.keys()):
    cont=0
    label = data[k]["frame_ann"]
    feat_p = data[k]["feat_p"]

    parts = feat_p.split(os.sep)
    if parts[0] not in folders:
        folders.append(parts[0])

    if label is not None:
        for j in range(len(label["labels"])):
            raw_label = label["labels"][j]["raw_label"]
            if raw_label is None:
                cont+=1
        if cont == len(label["labels"]):
            no_lab += 1
            continue

        occur_lab["label"].append(label)
        occur_lab["feat_p"].append(feat_p)
    else:
        no_lab += 1
            
print(len(folders), folders)
print("Dati totali: ", len(data))
print("Dati con label: ", len(occur_lab["label"]))
print("Dati senza label:", no_lab)
print("num label, paths:", len(occur_lab["label"]), len(occur_lab["feat_p"]))


17 ['BMLrub', 'ACCAD', 'CMU', 'MPIHDM05', 'EyesJapanDataset', 'KIT', 'EKUT', 'MPImosh', 'TCDhandMocap', 'DFaust67', 'MPILimits', 'SFU', 'TotalCapture', 'HumanEva', 'SSMsynced', 'BMLmovi', 'Transitionsmocap']
Dati totali:  10892
Dati con label:  5341
Dati senza label: 5551
num label, paths: 5341 5341


In [5]:

directory = '/home/michieletto/datasets/AMASS_H/ACCAD'
file_paths = glob.glob(directory + '/**/*.npz', recursive=True) # ottiene tutti i file nelle diverse cartelle contenenti i mocap in una stringa

directory = '/home/michieletto/datasets/AMASS_H/KIT'
file_paths += glob.glob(directory + '/**/*.npz', recursive=True)

directory = '/home/michieletto/datasets/AMASS_H/BMLrub'
file_paths += glob.glob(directory + '/**/*.npz', recursive=True)

directory = '/home/michieletto/datasets/AMASS_H/CMU'
file_paths += glob.glob(directory + '/**/*.npz', recursive=True)

directory = '/home/michieletto/datasets/AMASS_H/MPIHDM05'
file_paths += glob.glob(directory + '/**/*.npz', recursive=True)

directory = '/home/michieletto/datasets/AMASS_H/EyesJapanDataset'
file_paths += glob.glob(directory + '/**/*.npz', recursive=True)

directory = '/home/michieletto/datasets/AMASS_H/EKUT'
file_paths += glob.glob(directory + '/**/*.npz', recursive=True)

directory = '/home/michieletto/datasets/AMASS_H/MPImosh'
file_paths += glob.glob(directory + '/**/*.npz', recursive=True)

directory = '/home/michieletto/datasets/AMASS_H/TCDhandMocap'
file_paths += glob.glob(directory + '/**/*.npz', recursive=True)

directory = '/home/michieletto/datasets/AMASS_H/DFaust67'
file_paths += glob.glob(directory + '/**/*.npz', recursive=True)

directory = '/home/michieletto/datasets/AMASS_H/MPILimits'
file_paths += glob.glob(directory + '/**/*.npz', recursive=True)

directory = '/home/michieletto/datasets/AMASS_H/SFU'
file_paths += glob.glob(directory + '/**/*.npz', recursive=True)

directory = '/home/michieletto/datasets/AMASS_H/TotalCapture'
file_paths += glob.glob(directory + '/**/*.npz', recursive=True)

directory = '/home/michieletto/datasets/AMASS_H/HumanEva'
file_paths += glob.glob(directory + '/**/*.npz', recursive=True)

directory = '/home/michieletto/datasets/AMASS_H/SSMsynced'
file_paths += glob.glob(directory + '/**/*.npz', recursive=True)

directory = '/home/michieletto/datasets/AMASS_H/BMLmovi'
file_paths += glob.glob(directory + '/**/*.npz', recursive=True)

directory = '/home/michieletto/datasets/AMASS_H/Transitionsmocap'
file_paths += glob.glob(directory + '/**/*.npz', recursive=True)

file_paths = sorted(file_paths)
print(len(file_paths))

deleted_paths = []
for i in range(len(file_paths)):
    file_name = os.path.basename(file_paths[i])
    folder_path = os.path.dirname(file_paths[i])   # ottiene il percorso fino alla cartella in cui è contenuto il file
    last_folder_name = os.path.basename(folder_path)     # ottiene il nome dell'ultima cartella
    last_folder_and_file_name = os.path.join('/', last_folder_name, file_name)     # unisce i due nomi
    
    if file_name == "shape.npz":
        deleted_paths.append(i)
    else:
        for j in range(len(occur_lab["feat_p"])):
            if last_folder_and_file_name not in occur_lab["feat_p"][j] and j == len(occur_lab["feat_p"])-1:
                deleted_paths.append(i)
            elif last_folder_and_file_name in occur_lab["feat_p"][j]:
                break


for i in sorted(deleted_paths, reverse=True):
    del file_paths[i]

print(len(file_paths))

# -- load mocap --
mocap_loader=MocapLoader(body_model_type=SmplConstants.BODY_MODEL_TYPE,
                         keypoint_ids_to_load=SmplConstants.KEYPOINTS,
                         target_framerate = 25)

motions = []
motions = [mocap_loader.load_mocap(path) for path in file_paths] 


12728
4991


In [6]:
frame_vocab = []
deleted_paths = []

cont = 0
total_frames = 75
frames_to_jump = 10
window_frames = 50
for i in range(len(motions)):
    if motions[i] is not None:
        motions[i].rots = motions[i].rots[frames_to_jump:] # elimina i primi frame
        if len(motions[i].rots) >= total_frames:
            sample = []
            valid_mocap_frames = len(motions[i].rots)
            for j in range(int((valid_mocap_frames - 25)/window_frames)):
                sample.append(motions[i].rots[window_frames*j:window_frames*j + total_frames])
                assert len(sample[-1]) == 75

                sample[-1] = [[angle for k_point in frame for angle in k_point] for frame in sample[-1]]
                assert len(sample[-1][0]) == 72

            frame_vocab.append(sample)
        else:
            frame_vocab.append([])
            deleted_paths.append(i)
            cont +=1
    else:
        frame_vocab.append([])
        deleted_paths.append(i)
        cont += 1

In [7]:
print(f"Numero di mocap non validi: {cont}")

sum = 0
for i in range(len(frame_vocab)):
    sum += len(frame_vocab[i])
    
print(f"Il numero totale di mocap è: {sum}")

Numero di mocap non validi: 180
Il numero totale di mocap è: 32455


In [8]:
dataset = {
    'pose': [],
    'description': [],
}

for i in range(len(file_paths)):
    if i not in deleted_paths:
        filepath = file_paths[i]
        file_name = os.path.basename(filepath)    # ottiene il nome del file
        folder_path = os.path.dirname(filepath)   # ottiene il percorso fino alla cartella in cui è contenuto il file
        last_folder_name = os.path.basename(folder_path)     # ottiene il nome dell'ultima cartella
        last_folder_and_file_name = os.path.join('/', last_folder_name, file_name)     # unisce i due nomi
        
        for j in range(len(occur_lab["feat_p"])):
            if last_folder_and_file_name in occur_lab["feat_p"][j]:
                dataset["pose"].append(frame_vocab[i])
                dataset["description"].append(occur_lab["label"][j])

print(len(dataset['pose']), len(dataset['description']))

sum = 0
for i in range(len(dataset['pose'])):
    sum += len(dataset['pose'][i])
    
print(f"Il numero di mocap del dataset è: {sum}")


4811 4811
Il numero di mocap del dataset è: 32455


In [9]:
descriptions = []
min_window = 5
framerate = 25
for i in range(len(dataset["description"])):
    frames_descriptions = []
    labels = dataset["description"][i]["labels"]
    
    # crea la lista di tutte le label annotation trovate con più di 5 frame
    for j in range(len(labels)):
        annotation_list = []
        start_t = dataset["description"][i]["labels"][j]["start_t"]
        start_t = int(start_t*framerate)
        end_t = dataset["description"][i]["labels"][j]["end_t"]
        end_t = int(end_t*framerate)

        if end_t < start_t:
            end_t, start_t = start_t, end_t

        if start_t == end_t:
            continue

        annotation_list.append(start_t)
        annotation_list.append(end_t)

        raw_label = dataset["description"][i]["labels"][j]["raw_label"]
        if raw_label is not None:
            annotation_list.append(raw_label)
        else:
            # non entra mai qui, ma nel dubbio lo lascio
            annotation_list.append("unknown")

        frames_descriptions.append(annotation_list)
    
    # ordina la lista in ordeine crescente rispetto al frame di inizio
    frames_descriptions = sorted(frames_descriptions, key=lambda x: x[0])

    # se il frame più alto non corrisponde al massimo frame del mocap, lo si fa corrispondere
    end_frames = [row[1] for row in frames_descriptions]
    max_value = max(end_frames)
    indices = [index for index, value in enumerate(end_frames) if value == max_value]
    if max_value < (len(dataset["pose"][i])*window_frames) + frames_to_jump + framerate:
        for idx in indices:
            frames_descriptions[idx][1] = (len(dataset["pose"][i])*window_frames) + frames_to_jump + framerate

    # corregge un paio di errori del dataset e si assicura un paio di condizioni
    max_frame = 0
    for l in range(len(frames_descriptions)):
        if l > 0:
            if frames_descriptions[l][1] == frames_descriptions[l-1][1]:
                frames_descriptions[l-1][1] = frames_descriptions[l][0]

        if max_frame < frames_descriptions[l][1]:
            max_frame = frames_descriptions[l][1]

        # assert frames_descriptions[l][0] >= frames_to_jump
        assert frames_descriptions[l][0] < frames_descriptions[l][1]
    
    # creazione matrice delle label
    matrix = [["" for _ in range(max_frame)] for _ in range(len(frames_descriptions))]

    for m in range(len(frames_descriptions)):
        for col in range(frames_descriptions[m][0], frames_descriptions[m][1]):
            matrix[m][col] = frames_descriptions[m][2]
    
    num_columns = len(matrix[0])
    # Verifica quando si ha un cambio di colonna
    flag_column = [False] * num_columns
    
    for col in range(1, num_columns):
        column = [row[col] for row in matrix]
        prev_column = [row[col-1] for row in matrix]
        if column == prev_column:
            flag_column[col] = False
        else:
            flag_column[col] = True
    
    flag_column[frames_to_jump] = True
    start_frame = frames_to_jump
    end_frame = start_frame + total_frames
    mocap_descriptions = []
    while (end_frame <= num_columns):
        sentence = ""
        for col in range(start_frame, end_frame):
            if any(flag_column[col+1:col+min_window+1]) or end_frame - col <= min_window:
                result = True
            else:
                result = False
            if flag_column[col] == True and not result:
                phrase = ""
                for row in matrix:
                    if row[col] != "":
                        if phrase == "":
                            phrase = row[col]
                        else:
                            phrase = phrase + " and "+ row[col]
                if phrase == "":
                    phrase = "unknown"
                sentence = sentence + phrase + " then "
        if end_frame < num_columns - 1:
            flag_column[start_frame + window_frames + 1] = True
        if sentence[-6:] == " then ":
            sentence = sentence[:-6]

        start_frame += window_frames
        end_frame = start_frame + total_frames

        if sentence != "":
            mocap_descriptions.append(sentence)

    assert len(mocap_descriptions) == len(dataset["pose"][i])    
    descriptions.append(mocap_descriptions)

dataset["description"] = descriptions


In [10]:

kine = ForwardKinematics(SmplConstants.OFFSETS, SmplConstants.PARENTS)
kine.set_body_model_type(SmplConstants.BODY_MODEL_TYPE)

for i in range(10):
    continue
    rand_x = random.randint(0, len(dataset["pose"])-1)
    rand_y = random.randint(0, len(dataset["pose"][rand_x])-1)

    print(rand_x, rand_y)

    motion = dataset["pose"][rand_x][rand_y]
    motion = np.array(motion)
    motion = motion.reshape(75, 24, 3)

    keypoints_positions = kine.compute_fk(motion, apply_root_rotation=True)

    print(dataset["description"][rand_x][rand_y])

    motions_list = [keypoints_positions]
    subplot_ids = [0]
    subplot_titles = [dataset["description"][rand_x][rand_y]]
    camera_orientations = [CameraOrientation(100, 20)]
    pose_animation(motions_list, SmplConstants.PARENTS, animation_framerate = 25, subplot_ids=subplot_ids, 
                camera_orientations=camera_orientations, subplot_titles=subplot_titles,
                fig_opts={'dpi':90}, animation_mode=AnimationMode.HTML, gif_savepath=f'./Frame_ann_gif/{dataset["description"][rand_x][rand_y]}.gif')

In [11]:
print(len(dataset["description"]), len(dataset["pose"]))

dataset["description"] = list(itertools.chain.from_iterable(dataset["description"]))
dataset["pose"] = list(itertools.chain.from_iterable(dataset["pose"]))

print(len(dataset["description"]), len(dataset["pose"]))

dataset = pd.DataFrame(dataset)
dataset_copy = dataset.copy()

4811 4811
32455 32455


In [12]:
dataset = pd.DataFrame(dataset)
dataset_copy = dataset.copy()

In [13]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('KennethEnevoldsen/dfm-sentence-encoder-small-v1')
dataset["description"] = model.encode(dataset["description"])
print(dataset["description"].shape)


/home/michieletto/venv_pmerc/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(32455,)


In [25]:
class CustomDataset(Dataset):
    def __init__(self, tensor1, tensor2):
        self.tensor1 = tensor1
        self.tensor2 = tensor2

    def __len__(self):
        return len(self.tensor1)

    def __getitem__(self, idx):
        return self.tensor1[idx], self.tensor2[idx]

def get_collate_fn():
    def collate_fn(batch):
        batch["pose"] = [[row[i] for row in batch["pose"]] for i in range(len(batch["pose"][0]))]
        # batch["pose"] = np.array(batch["pose"])
        batch["pose"] = torch.tensor(batch["pose"])
        batch["pose"] = batch["pose"].float()
        
        return batch

    return collate_fn

def get_data_loader(ds, batch_size, shuffle=True):
    pose = (torch.tensor(np.array(ds["pose"]))).float()
    desc = (torch.tensor(np.array(ds["description"]))).float()
    class_set = CustomDataset(pose, desc)

    collate_fn = get_collate_fn()

    data_loader = DataLoader(
        dataset=class_set,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    
    return data_loader

batch_size = 64
RANDOM_SEED = 42

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=RANDOM_SEED)
valid_data, test_data = train_test_split(test_data, test_size=0.5, random_state=RANDOM_SEED)

train_data_loader = get_data_loader(train_data, batch_size)
valid_data_loader = get_data_loader(valid_data, batch_size)
test_data_loader = get_data_loader(test_data, batch_size)

batch = next(iter(train_data_loader))


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [15]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(self, tensor1, tensor2):
        self.tensor1 = tensor1
        self.tensor2 = tensor2

    def __len__(self):
        return len(self.tensor1)

    def __getitem__(self, idx):
        return self.tensor1[idx], self.tensor2[idx]
# Creazione di un esempio di DataFrame
data = {'col1': [1, 2, 3, 4, 5], 'col2': [6, 7, 8, 9, 10]}
# data = pd.DataFrame(data)
data["col1"] = torch.tensor(data["col1"])
data["col2"] = torch.tensor(data["col2"])
daaa = CustomDataset(data["col1"], data["col2"])
print(type(data["col1"]))
data_loader = DataLoader(
        dataset=daaa,
        batch_size=batch_size,
        shuffle=True,
    )

next(iter(data_loader))

<class 'torch.Tensor'>


[tensor([5, 1, 4, 2, 3]), tensor([10,  6,  9,  7,  8])]